In [1]:
import dash
import dash_bootstrap_components as dbc
import numpy as np
import plotly.graph_objs as go
from dash import Input, Output, dcc, html

In [2]:
# No funcionaba, así que cambié la versión de Flask: 
# pip install Flask==2.1.0

In [3]:
# Leemos las bases de datos
df_airports = pd.read_parquet("../EDA/df_airports.parquet")
# df_flights = pd.read_parquet("../EDA/df_flights.parquet")
# df_airports_date = pd.read_parquet("../EDA/df_airports_date.parquet")

In [4]:
df_airports.head()

,ORIGIN_AIRPORT,ORIGIN_AIRPORT_NAME,ORIGIN_CITY,ORIGIN_STATE,DEPARTURE_DELAY,ELAPSED_TIME,DISTANCE,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,...,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,OTHER_DELAY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DESTINATION_LATITUDE,DESTINATION_LONGITUDE,FLIGHTS,DELAYED_FLIGHTS,DELAYED_PERCENTAGE
0,ATL,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,8.850270,114.866383,640.178136,2.416871,1.785697,0.004062,...,2.878495,0.839232,1.022221,33.64044,-84.42694,35.552319,-86.109114,375903,122213,0.325118
1,ORD,Chicago O'Hare International Airport,Chicago,IL,13.205020,127.695187,721.919227,7.608336,3.284368,0.009161,...,5.254136,1.396482,1.102350,41.97960,-87.90446,38.347093,-90.637218,303888,121696,0.400463
2,DFW,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,11.057457,130.859005,788.274763,6.425772,2.247282,0.022907,...,4.349548,1.121342,1.149282,32.89595,-97.03720,35.462532,-95.100926,252938,98565,0.389680
3,DEN,Denver International Airport,Denver,CO,11.259532,137.207102,877.915006,6.472926,2.666673,0.004362,...,5.082969,0.446868,1.218958,39.85841,-104.66700,38.010582,-100.578165,211369,86012,0.406928
4,LAX,Los Angeles International Airport,Los Angeles,CA,10.173233,179.770106,1253.742786,5.286918,2.241610,0.016639,...,4.982808,0.162934,1.302307,33.94254,-118.40807,36.582438,-104.603276,209631,85590,0.408289


In [5]:
# Map chart
fig_map = px.scatter_geo(df_airports, lat="ORIGIN_LATITUDE", lon = "ORIGIN_LONGITUDE",
                     size= "FLIGHTS", # size of markers
                     size_max= 30,
                     color= "DELAYED_PERCENTAGE", # which column to use to set the color of markers
                     scope="usa",
                     text = "ORIGIN_AIRPORT",
                     hover_data  = ["ORIGIN_CITY"],
                     color_continuous_scale='RdYlGn_r',
                     template="plotly_dark")
fig_map.update_traces(textposition="top center")
fig_map.update_layout(
    title="Origin airports with number of departing flights and percentage of delayed flights <br><br><sup>Size indicates the number of departing flights</sup>",
    legend_title="Causa del Retraso",
)

In [6]:
app = dash.Dash(external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('AIRPORTS ANALYSIS', className='text-center text-primary, mb-3'))),  # header row
        
        dbc.Row([  # start of second row
            dbc.Col([  # first column on second row
            html.H5('Map', className='text-center'),
            dcc.Graph(id='map-main',
                      figure = fig_map,
                      hoverData={"points": [{"text": "ATL"}]},
                      style={'height':550}),
            html.Hr(),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on third row
                html.H5('Distribution', className='text-center'),
                dcc.Graph(id='pie-main',
                      style={'height':550}),
            ], width={'size': 4, 'offset': 0, 'order': 2}),  # width second column on second row
        ]),  # end of second row      
    ], fluid=True)

@app.callback(
    Output('pie-main', 'figure'),
    Input('map-main', 'hoverData'))
def update_pie(hoverdata):
    # Pie Chart
    airport = hoverdata['points'][0]['text']
    delay_labels = ["AIR_SYSTEM_DELAY","SECURITY_DELAY","AIRLINE_DELAY","LATE_AIRCRAFT_DELAY","WEATHER_DELAY","OTHER_DELAY"]
    values1 = df_airports[delay_labels].iloc[df_airports[df_airports["ORIGIN_AIRPORT"]==airport].index[0]]
    fig_pie = go.Figure()
    fig_pie.add_trace(go.Pie(labels=delay_labels, values=values1, direction ='clockwise', marker_colors=px.colors.qualitative.Vivid, 
                                hole=.3, title = airport))
    fig_pie.update_layout(title_text="Average Delay Distribution by Airport", legend_title="Delay Cause", template="plotly_dark",
                        legend=dict(orientation="h", y=-0.02, x =0.08))
    return fig_pie

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [05/Dec/2022 01:11:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 01:11:52] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 01:11:52] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 01:11:53] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 01:11:53] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2022 01:11:53] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [05/Dec/2022 01:11:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 01:11:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 01:11:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 01:11:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2022 01:12:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2